In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import os
import sqlite3

In [ ]:
df = pd.read_csv("../Motor_Vehicle_Collisions_-_Crashes_20251015.csv")
df

# Lidando com Missing Values

In [ ]:
df.isnull().sum().sum() # Muitos valores nulos impossivel exclui-los

## ~~Removendo Colunas com dados nulos acima de 82%~~

In [ ]:
df.isna().sum() # colunas off street name, contributing factor 3 veicle ate o 5 , vehicle type code 3 ate o 5 sao colunas com muitos valores nulos e podem ser removidas
missing_percent = df.isna().mean() * 100
print(missing_percent.sort_values(ascending=False))
# df_removed = df.drop(columns=['OFF STREET NAME', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'])
# df_removed.isna().sum().sum()

In [ ]:
# comparando antes e depois de remover os nulos
# print(df.shape)
# print(df_removed.shape)

In [ ]:
# # demonstração de valores faltantes em ordem descendente
# df_removed.isna().sum() 
# missing_percent = df_removed.isna().mean() * 100
# print(missing_percent.sort_values(ascending=False))

# Lidando Com Outliers

In [ ]:
# Converte os outliars de longitude e latitude em Nan
lat_min, lat_max = 40.49, 40.92
lon_min, lon_max = -74.26, -73.69
filtro_invalidas = (
    (df['LATITUDE'] < lat_min) | (df['LATITUDE'] > lat_max) |
    (df['LONGITUDE'] < lon_min) | (df['LONGITUDE'] > lon_max)
)
df.loc[filtro_invalidas, ['LATITUDE', 'LONGITUDE']] = np.nan

## Usando novo dataset sem colunas com muitos nulos para imputar restante dos resultados com simpleImputer 

In [ ]:
num_cols_auto = df.select_dtypes(include=['number']).columns.to_list()
cat_cols_auto = df.select_dtypes(include=['object']).columns.to_list()
num_cols_imputar = [
    col for col in num_cols_auto 
    if col not in ['LATITUDE', 'LONGITUDE', 'COLLISION_ID'] 
]
cat_cols_imputar = cat_cols_auto


In [ ]:
transformers_imput = [
    ('num_imputer', SimpleImputer(strategy='median'), num_cols_imputar),
    ('freq_imputer', SimpleImputer(strategy='most_frequent'), cat_cols_imputar)
]
ct_imput = ColumnTransformer(transformers_imput, remainder='passthrough')
df_imputed_array = ct_imput.fit_transform(df)

# Obter nomes das colunas 'remainder'
imputed_cols = num_cols_imputar + cat_cols_imputar
all_cols_original = df.columns.tolist()
remainder_cols = [col for col in all_cols_original if col not in imputed_cols]

# Ordem correta das colunas no array
nova_ordem_colunas = num_cols_imputar + cat_cols_imputar + remainder_cols

# Reconstruir
df_imputed = pd.DataFrame(df_imputed_array, columns=nova_ordem_colunas)

colunas_numericas_finais = num_cols_imputar + remainder_cols

for col in colunas_numericas_finais:
    # pd.to_numeric é a melhor forma de fazer isso
    df_imputed[col] = pd.to_numeric(df_imputed[col], errors='coerce')

# 2. (Opcional) Garante que as colunas categóricas sejam 'string'
for col in cat_cols_imputar:
    df_imputed[col] = df_imputed[col].astype(str)

print("\n--- DEPOIS da Conversão de Tipo (Tipos Corrigidos) ---")
df_imputed.info()

In [ ]:
df_imputed.info()

In [ ]:
df_imputed.isna().sum() 
missing_percent = df_imputed.isna().mean() * 100
print(missing_percent.sort_values(ascending=False))

# Resolvendo inconsistências

In [ ]:
try:
    # df = pd.read_csv('df_8_14_colunas.csv')
    # print(f"--- Carregado 'df_8_14_colunas.csv' para Identificação ---")
    
    # 1. Identificar todas as colunas que são do tipo 'object' (texto)
    object_cols = df_imputed.select_dtypes(include=['object']).columns
    
    if len(object_cols) == 0:
        print("Nenhuma coluna de texto (object) encontrada neste arquivo.")
    else:
        print(f"Colunas de texto (object) encontradas: {list(object_cols)}")
        
        # Limite para decidir se imprimimos todos os valores únicos
        low_cardinality_threshold = 50 
        
        print("\n--- 2. Análise de Valores Únicos (Identificação) ---")
        
        for col in object_cols:
            num_unique = df[col].nunique()
            
            print(f"\nColuna: '{col}' | Total de Valores Únicos: {num_unique}")
            
            # Se a coluna tiver poucos valores únicos, é categórica.
            # Vamos imprimir as contagens para o usuário ver.
            if num_unique <= low_cardinality_threshold:
                print("Contagem de valores (value_counts):")
                # Imprimir .value_counts() que já vem ordenado
                print(df[col].value_counts(dropna=False)) # dropna=False para vermos os NaNs
            else:
                print(f"Cardinalidade muito alta. Exibindo 5 valores de exemplo:")
                print(df[col].sample(5, random_state=1).to_list())
except FileNotFoundError:
    print("Erro: O arquivo 'df_8_14_colunas.csv' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

In [ ]:
df_reduzido = df_imputed.iloc[:, 14:16]
df_reduzido.to_csv("df_14_16_colunas.csv", index=False)

In [ ]:
df_imputed

# Resolvendo Tipos

In [ ]:
df_imputed.info()

In [ ]:
df_typed = df_imputed.copy()

# Convert all columns that start with "NUMBER" to int
number_cols = [col for col in df_typed.columns if col.startswith("NUMBER")]
for col in number_cols:
    df_typed[col] = pd.to_numeric(df_typed[col], errors='coerce').astype('Int64')




df_typed.info()

In [ ]:
# Check date-related columns
date_cols = [col for col in df_typed.columns if 'DATE' in col.upper() or 'TIME' in col.upper()]
print("Date/Time columns found:")
print(date_cols)
print("\nSample values:")
for col in date_cols:
    print(f"\n{col}:")
    print(df_typed[col].head())

In [ ]:
# Combine CRASH DATE and CRASH TIME into a single datetime column
df_typed['CRASH_DATETIME'] = pd.to_datetime(
    df_typed['CRASH DATE'] + ' ' + df_typed['CRASH TIME'], 
    format='%m/%d/%Y %H:%M',
    errors='coerce'
)

# Display the result
print("New CRASH_DATETIME column created:")
print(df_typed[['CRASH DATE', 'CRASH TIME', 'CRASH_DATETIME']].head(10))
print(f"\nData type: {df_typed['CRASH_DATETIME'].dtype}")
print(f"Null values: {df_typed['CRASH_DATETIME'].isna().sum()}")

# Drop the original date and time columns
df_typed = df_typed.drop(columns=['CRASH DATE', 'CRASH TIME'])
print("\n--- After dropping CRASH DATE and CRASH TIME ---")
print(df_typed.info())

In [ ]:
# Convert all object columns to explicit string type
object_cols = df_typed.select_dtypes(include=['object']).columns.tolist()
print(f"Converting {len(object_cols)} object columns to string type:")
print(object_cols)

for col in object_cols:
    df_typed[col] = df_typed[col].astype('string')

print("\n--- After converting objects to strings ---")
print(df_typed.info())

In [ ]:
#versao sem ID
df_typed = df_typed.drop(columns = ['LOCATION'])
df_noid = df_typed.drop(columns=['COLLISION_ID'])
print(df_noid)
df_noid.info()

# SQL

In [ ]:
# Recarregar dados brutos para comparação
# df_raw: dados originais para comparar (sem imputação/outlier fix)
df_raw = pd.read_csv("../Motor_Vehicle_Collisions_-_Crashes_20251015.csv")
print("Linhas raw:", len(df_raw))
print("Linhas clean (df_typed):", len(df_typed))

# Bounding box NYC utilizado na limpeza
lat_min, lat_max = 40.49, 40.92
lon_min, lon_max = -74.26, -73.69

# Marcar outliers nas coordenadas do raw (antes de limpeza)
raw_outlier_mask = (
    (df_raw['LATITUDE'] < lat_min) | (df_raw['LATITUDE'] > lat_max) |
    (df_raw['LONGITUDE'] < lon_min) | (df_raw['LONGITUDE'] > lon_max)
)
print("Outliers LAT/LON no raw:", raw_outlier_mask.sum())
print("NAN LAT/LON já no clean:", df_typed['LATITUDE'].isna().sum(), df_typed['LONGITUDE'].isna().sum())

## Resumo da Validação da Limpeza
Principais achados dos dados brutos vs dados limpos:

1. Valores Nulos:
   - Conjunto bruto apresenta ~10–20% de nulos em colunas secundárias (VEHICLE TYPE CODE 2, CONTRIBUTING FACTOR VEHICLE 2) e ~11% em LATITUDE/LONGITUDE (após marcação de inválidos).
   - Conjunto limpo mantém proporções controladas nas colunas chave e converte coordenadas inválidas explicitamente em NaN para tratamento downstream.
2. Outliers Geográficos:
   - Tínhamos 6.485 registros fora do bounding box (NYC). Visualização mostra pontos dispersos irreais (ex.: longitudes extremas). Após limpeza, distribuição concentra-se corretamente sobre NYC.
3. Tipagem e Consistência:
   - Colunas numéricas convertidas para tipos inteiros (nullable Int64) garantindo coerência para agregações.
   - Datas unificadas em `CRASH_DATETIME` permitindo análises temporais diretas (evolução mensal preservada).
4. Fatores Contributivos & Tipos de Veículo:
   - Ranking top de fatores foi preservado (ex.: Driver Inattention/Distraction, Failure to Yield). Heatmap evidencia relação fator x tipo: sedans e passenger vehicles predominam nos fatores inespecíficos e de distração.
5. Evolução Temporal:
   - Série mensal consistente entre cru e limpo, indicando que transformação não distorceu volume global por período; quedas visíveis relativas à pandemia permanecem.
6. Redução de Risco Analítico:
   - Remoção/isolamento de outliers e padronização de tipos reduz vieses em modelos futuros (ex.: clustering por área ou estimativa de severidade).

Conclusão: A limpeza aplicada melhora a confiabilidade espacial e de consistência de tipos/date-time sem alterar tendências globais; próximos passos podem incluir normalização semântica adicional (ex.: agrupar fatores raros em 'Other') e enriquecimento geográfico (reverse geocoding de LAT/LON válidos).


In [ ]:
# Criar conexão SQLite em memória e registrar tabelas reduzidas para eficiência

conn = sqlite3.connect(":memory:")

cols_interest = [
    'CRASH_DATETIME', 'LATITUDE', 'LONGITUDE',
    'NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED',
    'NUMBER OF PEDESTRIANS INJURED','NUMBER OF PEDESTRIANS KILLED',
    'NUMBER OF CYCLIST INJURED','NUMBER OF CYCLIST KILLED',
    'NUMBER OF MOTORIST INJURED','NUMBER OF MOTORIST KILLED',
    'CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2',
    'VEHICLE TYPE CODE 1','VEHICLE TYPE CODE 2'
]

# Preparar raw com datetime reconstruído
raw_dt = pd.to_datetime(df_raw['CRASH DATE'] + ' ' + df_raw['CRASH TIME'], errors='coerce')
df_raw_sql = df_raw.assign(CRASH_DATETIME=raw_dt)[cols_interest]

df_clean_sql = df_typed[cols_interest].copy()

# Registrar
%time df_raw_sql.to_sql('crashes_raw', conn, index=False)
%time df_clean_sql.to_sql('crashes_clean', conn, index=False)

print('Tabelas criadas: crashes_raw, crashes_clean')

In [ ]:
# Consultas SQL diagnósticas
queries = {
    'count_rows_raw': "SELECT COUNT(*) AS total_raw FROM crashes_raw;",
    'count_rows_clean': "SELECT COUNT(*) AS total_clean FROM crashes_clean;",
    'nulls_raw': """
        SELECT column_name,
               SUM(CASE WHEN value IS NULL THEN 1 ELSE 0 END) AS nulls,
               COUNT(*) AS total,
               ROUND(100.0*SUM(CASE WHEN value IS NULL THEN 1 ELSE 0 END)/COUNT(*),2) AS pct_null
        FROM (
          SELECT CRASH_DATETIME AS value, 'CRASH_DATETIME' AS column_name FROM crashes_raw UNION ALL
          SELECT LATITUDE,'LATITUDE' FROM crashes_raw UNION ALL
          SELECT LONGITUDE,'LONGITUDE' FROM crashes_raw UNION ALL
          SELECT `NUMBER OF PERSONS INJURED`,'NUMBER OF PERSONS INJURED' FROM crashes_raw UNION ALL
          SELECT `NUMBER OF PERSONS KILLED`,'NUMBER OF PERSONS KILLED' FROM crashes_raw UNION ALL
          SELECT `CONTRIBUTING FACTOR VEHICLE 1`,'CONTRIBUTING FACTOR VEHICLE 1' FROM crashes_raw UNION ALL
          SELECT `CONTRIBUTING FACTOR VEHICLE 2`,'CONTRIBUTING FACTOR VEHICLE 2' FROM crashes_raw UNION ALL
          SELECT `VEHICLE TYPE CODE 1`,'VEHICLE TYPE CODE 1' FROM crashes_raw UNION ALL
          SELECT `VEHICLE TYPE CODE 2`,'VEHICLE TYPE CODE 2' FROM crashes_raw
        ) t
        GROUP BY column_name ORDER BY pct_null DESC;""",
    'nulls_clean': """
        SELECT column_name,
               SUM(CASE WHEN value IS NULL THEN 1 ELSE 0 END) AS nulls,
               COUNT(*) AS total,
               ROUND(100.0*SUM(CASE WHEN value IS NULL THEN 1 ELSE 0 END)/COUNT(*),2) AS pct_null
        FROM (
          SELECT CRASH_DATETIME AS value, 'CRASH_DATETIME' AS column_name FROM crashes_clean UNION ALL
          SELECT LATITUDE,'LATITUDE' FROM crashes_clean UNION ALL
          SELECT LONGITUDE,'LONGITUDE' FROM crashes_clean UNION ALL
          SELECT `NUMBER OF PERSONS INJURED`,'NUMBER OF PERSONS INJURED' FROM crashes_clean UNION ALL
          SELECT `NUMBER OF PERSONS KILLED`,'NUMBER OF PERSONS KILLED' FROM crashes_clean UNION ALL
          SELECT `CONTRIBUTING FACTOR VEHICLE 1`,'CONTRIBUTING FACTOR VEHICLE 1' FROM crashes_clean UNION ALL
          SELECT `CONTRIBUTING FACTOR VEHICLE 2`,'CONTRIBUTING FACTOR VEHICLE 2' FROM crashes_clean UNION ALL
          SELECT `VEHICLE TYPE CODE 1`,'VEHICLE TYPE CODE 1' FROM crashes_clean UNION ALL
          SELECT `VEHICLE TYPE CODE 2`,'VEHICLE TYPE CODE 2' FROM crashes_clean
        ) t
        GROUP BY column_name ORDER BY pct_null DESC;""",
    'top_contributing_factors_raw': """
        SELECT `CONTRIBUTING FACTOR VEHICLE 1` AS factor, COUNT(*) AS cnt
        FROM crashes_raw
        WHERE `CONTRIBUTING FACTOR VEHICLE 1` IS NOT NULL AND TRIM(`CONTRIBUTING FACTOR VEHICLE 1`) <> ''
        GROUP BY factor ORDER BY cnt DESC LIMIT 15;""",
    'top_contributing_factors_clean': """
        SELECT `CONTRIBUTING FACTOR VEHICLE 1` AS factor, COUNT(*) AS cnt
        FROM crashes_clean
        WHERE `CONTRIBUTING FACTOR VEHICLE 1` IS NOT NULL AND TRIM(`CONTRIBUTING FACTOR VEHICLE 1`) <> ''
        GROUP BY factor ORDER BY cnt DESC LIMIT 15;""",
    'vehicle_types_raw': """
        SELECT `VEHICLE TYPE CODE 1` AS vtype, COUNT(*) AS cnt
        FROM crashes_raw
        WHERE `VEHICLE TYPE CODE 1` IS NOT NULL AND TRIM(`VEHICLE TYPE CODE 1`) <> ''
        GROUP BY vtype ORDER BY cnt DESC LIMIT 15;""",
    'vehicle_types_clean': """
        SELECT `VEHICLE TYPE CODE 1` AS vtype, COUNT(*) AS cnt
        FROM crashes_clean
        WHERE `VEHICLE TYPE CODE 1` IS NOT NULL AND TRIM(`VEHICLE TYPE CODE 1`) <> ''
        GROUP BY vtype ORDER BY cnt DESC LIMIT 15;""",
    'monthly_counts_raw': """
        SELECT STRFTIME('%Y-%m', CRASH_DATETIME) AS ym, COUNT(*) AS cnt
        FROM crashes_raw
        WHERE CRASH_DATETIME IS NOT NULL
        GROUP BY ym ORDER BY ym;""",
    'monthly_counts_clean': """
        SELECT STRFTIME('%Y-%m', CRASH_DATETIME) AS ym, COUNT(*) AS cnt
        FROM crashes_clean
        WHERE CRASH_DATETIME IS NOT NULL
        GROUP BY ym ORDER BY ym;"""
}

results = {}
for name, q in queries.items():
    results[name] = pd.read_sql_query(q, conn)
    print(f"--- {name} ---")
    print(results[name].head())
    print()

# Guardar para uso nas visualizações
monthly_raw = results['monthly_counts_raw']
monthly_clean = results['monthly_counts_clean']
nulls_raw = results['nulls_raw']
nulls_clean = results['nulls_clean']
veh_raw = results['vehicle_types_raw']
veh_clean = results['vehicle_types_clean']

In [ ]:

# Missing % comparação
miss_compare = nulls_raw[['column_name','pct_null']].merge(
    nulls_clean[['column_name','pct_null']], on='column_name', suffixes=('_raw','_clean')
)
plt.figure(figsize=(10,5))
miss_compare_melt = miss_compare.melt(id_vars='column_name', value_vars=['pct_null_raw','pct_null_clean'],
                                      var_name='dataset', value_name='pct_null')
sns.barplot(data=miss_compare_melt, x='column_name', y='pct_null', hue='dataset')
plt.xticks(rotation=45, ha='right')
plt.title('Percentual de Nulos: Raw vs Clean')
plt.tight_layout()
plt.show()

# Evolução mensal
plt.figure(figsize=(12,5))
plt.plot(monthly_raw['ym'], monthly_raw['cnt'], label='Raw', alpha=0.6)
plt.plot(monthly_clean['ym'], monthly_clean['cnt'], label='Clean', alpha=0.6)
plt.xticks(monthly_clean['ym'][::max(len(monthly_clean)//12,1)], rotation=45)
plt.title('Contagem Mensal de Colisões')
plt.legend()
plt.tight_layout()
plt.show()

# Top tipos de veículo antes/depois
veh_merge = veh_raw.merge(veh_clean, on='vtype', suffixes=('_raw','_clean'))
veh_merge['diff'] = veh_merge['cnt_clean'] - veh_merge['cnt_raw']
plt.figure(figsize=(10,6))
veh_merge_sorted = veh_merge.sort_values('cnt_clean', ascending=False).head(15)
sns.barplot(data=veh_merge_sorted.melt(id_vars='vtype', value_vars=['cnt_raw','cnt_clean']), x='vtype', y='value', hue='variable')
plt.xticks(rotation=45, ha='right')
plt.title('Top 15 Tipos de Veículo: Raw vs Clean')
plt.ylabel('Contagem')
plt.tight_layout()
plt.show()

# Coordenadas: distribuição (amostra para performance)
sample_raw = df_raw_sql[['LATITUDE','LONGITUDE']].dropna().sample(20000, random_state=42) if len(df_raw_sql)>20000 else df_raw_sql[['LATITUDE','LONGITUDE']].dropna()
sample_clean = df_clean_sql[['LATITUDE','LONGITUDE']].dropna().sample(20000, random_state=42) if len(df_clean_sql)>20000 else df_clean_sql[['LATITUDE','LONGITUDE']].dropna()

fig, axes = plt.subplots(1,2, figsize=(12,5))
axes[0].scatter(sample_raw['LONGITUDE'], sample_raw['LATITUDE'], s=3, alpha=0.3)
axes[0].set_title('Raw Coordenadas (amostra)')
axes[1].scatter(sample_clean['LONGITUDE'], sample_clean['LATITUDE'], s=3, alpha=0.3, color='green')
axes[1].set_title('Clean Coordenadas (amostra)')
for ax in axes:
    ax.set_xlabel('Longitude'); ax.set_ylabel('Latitude')
plt.tight_layout(); plt.show()

# Heatmap fatores x tipos (clean)
factor_type = pd.read_sql_query("""
    SELECT `CONTRIBUTING FACTOR VEHICLE 1` AS factor, `VEHICLE TYPE CODE 1` AS vtype
    FROM crashes_clean
    WHERE factor IS NOT NULL AND vtype IS NOT NULL AND TRIM(factor)<>'' AND TRIM(vtype)<>''
""", conn)
# limitar top para evitar matriz enorme
top_factors = factor_type['factor'].value_counts().head(10).index
top_vtypes = factor_type['vtype'].value_counts().head(10).index
matrix_df = factor_type[factor_type['factor'].isin(top_factors) & factor_type['vtype'].isin(top_vtypes)]
pivot = matrix_df.pivot_table(index='factor', columns='vtype', aggfunc=len, fill_value=0)
plt.figure(figsize=(10,6))
sns.heatmap(pivot, cmap='Blues')
plt.title('Heatmap Fatores vs Tipos (Top10) - Clean')
plt.tight_layout(); plt.show()